In [ ]:
import os
import torch
import torch.nn as nn # Neural Network
import torch.optim as optim # Optimizer's Relu etc. 
from torch.utils.data import Dataset, DataLoader # Preprocessing Dataset, Loading Dataset

from PIL import Image # Open Image
import torchvision.transforms as transforms # Image Processing & Augmentation i.e. Resize, Flip, Normalize etc. 
    
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity # Measures Similarity Between Vectors
from sklearn.preprocessing import MultiLabelBinarizer

# Device Configuration

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Loading the Dataset

In [ ]:
import pandas as pd

movies = pd.read_csv('/kaggle/input/movie-recommendation-system/movies.csv')

ratings = pd.read_csv('/kaggle/input/movie-recommendation-system/ratings.csv')

movies.head()

# Genre Preprocessing

## Convert genre string → list

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))

## Multi-Hot Encode genres

In [ ]:
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movies['genres'])

genre_matrix.shape

# Pytorch Dataset

In [ ]:
class MovieDataset(Dataset):
    def __init__(self, genre_matrix):
        self.X = torch.tensor(genre_matrix, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.X[idx]  # autoencoder-style

# CNN Model From Scratch

## 1D CNN Autoencoder

In [ ]:
class GenreCNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )

        self.fc = nn.Linear(64, input_dim)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch, channel, features)
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Training Setup

In [ ]:
dataset = MovieDataset(genre_matrix)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GenreCNN(genre_matrix.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Model Training

In [ ]:
epochs = 10

for epoch in range(epochs):
    total_loss = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")

# Generate Movie Embeddings

In [ ]:
model.eval()
with torch.no_grad():
    embeddings = model(torch.tensor(genre_matrix, dtype=torch.float32).to(device))
    embeddings = embeddings.cpu().numpy()

# Similarity Function (Core Recommendation Logic

In [ ]:
def recommend_similar_movies(movie_id, top_n=10):
    idx = movies[movies['movieId'] == movie_id].index[0]

    similarity = cosine_similarity(
        embeddings[idx].reshape(1, -1),
        embeddings
    )[0]

    similar_indices = similarity.argsort()[::-1][1:top_n+1]

    return movies.iloc[similar_indices][['movieId', 'title']]

# Test Recommendation:

In [ ]:
recommend_similar_movies(movie_id=1, top_n=10)

# Personalized Recommendation:

In [ ]:
def recommend_for_user(user_id, top_n=10):
    user_ratings = ratings[ratings['userId'] == user_id]
    liked = user_ratings[user_ratings['rating'] >= 4]['movieId']

    liked_indices = movies[movies['movieId'].isin(liked)].index
    user_vector = embeddings[liked_indices].mean(axis=0)

    similarity = cosine_similarity(user_vector.reshape(1, -1), embeddings)[0]
    watched = set(liked_indices)

    recs = [i for i in similarity.argsort()[::-1] if i not in watched][:top_n]

    return movies.iloc[recs][['movieId', 'title']]

# Run Personalized Recommendation

In [ ]:
recommend_for_user(user_id=10)

# Install IPWidget

In [ ]:
!pip install ipywidgets

# import ipywidgets

In [ ]:
import ipywidgets:
import ipywidgets as widgets
from IPython.display import display, clear_outpu

# Mapping Movie Data

In [ ]:
movie_titles = movies[['movieId', 'title']]
movie_id_to_index = {mid: idx for idx, mid in enumerate(movies['movieId'])}
title_to_movie_id = dict(zip(movies['title'], movies['movieId']))

# Showing Movies in Dropdown

In [ ]:
movie_dropdown = widgets.Dropdown(
    options=sorted(movies['title'].tolist()),
    description='Movie:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

recommend_button = widgets.Button(
    description="Recommend Similar Movies",
    button_style='success'
)

output = widgets.Output()

In [ ]:
def on_recommend_clicked(b):
    with output:
        clear_output()
        selected_title = movie_dropdown.value
        movie_id = title_to_movie_id[selected_title]

        recs = recommend_similar_movies(movie_id, top_n=10)
        display(recs)

# Recommend button

In [ ]:
recommend_button.on_click(on_recommend_clicked)

In [ ]:
display(movie_dropdown, recommend_button, output)

# User Based Recommendation

In [ ]:
user_input = widgets.IntText(
    value=1,
    description='User ID:',
    style={'description_width': 'initial'}
)

user_button = widgets.Button(
    description="Get Personalized Recommendations",
    button_style='info'
)

user_output = widgets.Output()

In [ ]:
def on_user_recommend_clicked(b):
    with user_output:
        clear_output()
        try:
            recs = recommend_for_user(user_input.value, top_n=10)
            display(recs)
        except:
            print("Invalid user ID or no ratings available.")

# Button to Show Recommedation

In [ ]:
user_button.on_click(on_user_recommend_clicked)

In [ ]:
display(user_input, user_button, user_output)